In [1]:
import json
from os import path
import instructor
from anthropic import Anthropic
from pydantic import BaseModel
from typing import Literal

def get_json_file(file_path: str) -> dict:
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        raise FileNotFoundError(f"The file {file_path} was not found")
    except json.JSONDecodeError:
        raise json.JSONDecodeError(f"The file {file_path} is not valid JSON")

def _get_project_dir_folder():
    return path.dirname(path.abspath("."))

In [2]:
config_fp = path.join(_get_project_dir_folder(), "assets", "config.json")
"""
Returns
    {"k" : "claude-api-key"}
"""
config = get_json_file(config_fp)

In [3]:
class GeneratedReview(BaseModel):
    review: str
    sentiment: Literal["positive", "negative"]

class SentimentPrediction(BaseModel):
    sentiment: Literal["positive", "negative"]

In [4]:
def generate_sample(client, input_review, sentiment):
    return client.chat.completions.create(
        model="claude-3-opus-20240229",
        response_model=GeneratedReview,
        messages=[
            {
                "role": "user",
                "content": f"""
                           Generate a '{sentiment}' review similar to: {input_review}
                           Generated review:
                           """,
            }
        ],
        max_tokens=1000
        
    )
def predict_sentiment(client, input_review):
    return client.chat.completions.create(
        model="claude-3-opus-20240229",
        response_model=SentimentPrediction,
        messages=[
            {
                "role": "user",
                "content": f"""Identify the sentiment of the following review.
                Review: {input_review}
                Sentiment:
                """,
            }
        ],
        max_tokens=1000
    )

In [5]:
client = instructor.from_anthropic(Anthropic(api_key=config.get("k")))

input_review = (
    "This movie was a rollercoaster of emotions, keeping me engaged throughout."
)
sentiment = 'positive'
review = generate_sample(client, input_review, sentiment)

In [6]:
review.review

'This film was a captivating journey from start to finish, never letting up on the intensity and drawing me in with its intricate plot and masterful performances.'

In [7]:
review.sentiment

'positive'

In [8]:
output_sentiment = predict_sentiment(client, review.review)

In [9]:
output_sentiment

SentimentPrediction(sentiment='positive')

In [10]:
output_sentiment.sentiment

'positive'